# 📋 Padrões GoF no LangExtract

## 🎯 5 Padrões Identificados

| # | Padrão | Arquivo | GoF | O que faz |
|---|--------|---------|-----|-----------|
| 1 | **Factory** | `factory.py` | ⚠️ | Cria providers (Gemini/OpenAI/Ollama) via `create_model()` |
| 2 | **Iterator** | `chunking.py` | ✅ | Itera documentos grandes com `ChunkIterator.__next__()` |
| 3 | **Facade** | `extraction.py` | ✅ | Função `extract()` oculta 6 subsistemas |
| 4 | **Builder** | `prompting.py` | ⚠️ | `QAPromptGenerator.render()` constrói prompts em 6 passos |
| 5 | **Strategy** | `base_model.py` | ✅ | `BaseLanguageModel` interface, 3 algoritmos (Gemini/OpenAI/Ollama) |

**Legenda:** ✅ 100% GoF | ⚠️ Variação simplificada

---

## 📝 Evidências

### 1. Factory (`factory.py`)
```python
def create_model(config) -> BaseLanguageModel:
    provider_cls = router.resolve(config.model_id)
    return provider_cls(**kwargs)  # Retorna Gemini/OpenAI/Ollama
```

### 2. Iterator (`chunking.py`)
```python
class ChunkIterator:
    def __next__(self) -> Chunk:
        if not self._has_more:
            raise StopIteration
        return self._get_next_chunk()
```

### 3. Facade (`extraction.py`)
```python
def extract(...):
    model = create_model(...)      # Factory
    prompt = QAPromptGenerator()   # Builder
    return Annotator(model).annotate(...)  # Strategy
```

### 4. Builder (`prompting.py`)
```python
def render(question):
    lines = []
    lines.append(description)       # Passo 1
    lines.append(context)           # Passo 2
    lines.append(examples)          # Passo 3
    lines.append(question)          # Passo 4
    return "\n".join(lines)
```

### 5. Strategy (`base_model.py` + providers)
```python
class BaseLanguageModel(ABC):
    @abstractmethod
    def infer(...): pass

class GeminiLanguageModel(BaseLanguageModel):
    def infer(...): # google-genai SDK

class OpenAILanguageModel(BaseLanguageModel):
    def infer(...): # openai SDK
```

---

## � Documentação Detalhada

1. `factory_pattern_analysis.md`
2. `iterator_pattern_analysis.md`
3. `facade_pattern_analysis.md`
4. `builder_pattern_analysis.md`
5. `strategy_pattern_analysis.md`

**Total:** 5 padrões GoF com comparações código clássico vs LangExtract.


# 🏭 Factory Pattern - Análise Comparativa
## LangExtract vs Padrão Arquitetural Clássico

---

## 📚 **1. PADRÃO FACTORY CLÁSSICO (GoF)**

### **Definição Teórica**

> **Factory Method Pattern**: Define uma interface para criar um objeto, mas deixa as subclasses decidirem qual classe instanciar. O Factory Method permite que uma classe delegue a instanciação para subclasses.

### **Estrutura UML Clássica**

```
┌─────────────────────┐
│    Creator          │ (Abstract)
├─────────────────────┤
│ + factoryMethod()   │ ◄─── Método abstrato
│ + operation()       │
└─────────────────────┘
         △
         │ (herança)
         │
┌────────┴────────────┐
│ ConcreteCreatorA    │
├─────────────────────┤
│ + factoryMethod()   │ ◄─── Retorna ProductA
└─────────────────────┘

┌─────────────────────┐
│    Product          │ (Interface)
└─────────────────────┘
         △
         │
┌────────┴────────────┬────────────────┐
│   ProductA          │   ProductB     │
└─────────────────────┴────────────────┘
```

### **Código Exemplo Clássico**

```python
from abc import ABC, abstractmethod

# 1. PRODUCT - Interface comum
class Animal(ABC):
    @abstractmethod
    def speak(self) -> str:
        pass

# 2. CONCRETE PRODUCTS - Implementações específicas
class Dog(Animal):
    def speak(self) -> str:
        return "Woof!"

class Cat(Animal):
    def speak(self) -> str:
        return "Meow!"

# 3. CREATOR - Factory abstrata
class AnimalFactory(ABC):
    @abstractmethod
    def create_animal(self) -> Animal:
        """Factory Method - subclasses implementam"""
        pass
    
    def make_sound(self) -> str:
        """Operação que usa o factory method"""
        animal = self.create_animal()
        return animal.speak()

# 4. CONCRETE CREATORS - Factories específicas
class DogFactory(AnimalFactory):
    def create_animal(self) -> Animal:
        return Dog()

class CatFactory(AnimalFactory):
    def create_animal(self) -> Animal:
        return Cat()

# 5. USO
factory = DogFactory()
print(factory.make_sound())  # Output: Woof!
```

---

## 🔍 **2. FACTORY NO LANGEXTRACT (factory.py)**

### **Estrutura do Código Real**

```python
# ============================================
# ARQUIVO: langextract/factory.py
# ============================================

import dataclasses
from langextract import providers
from langextract.providers import router

@dataclasses.dataclass(slots=True, frozen=True)
class ModelConfig:
    """Configuração para criar um provider"""
    model_id: str | None = None
    provider: str | None = None
    provider_kwargs: dict[str, Any] = dataclasses.field(default_factory=dict)

# ============================================
# FACTORY METHOD - Função principal
# ============================================
def create_model(
    config: ModelConfig,
    examples: Sequence[Any] | None = None,
    use_schema_constraints: bool = False,
    fence_output: bool | None = None,
) -> BaseLanguageModel:
    """
    Factory Method que cria instâncias de providers.
    
    Equivale ao método abstrato factoryMethod() do padrão GoF.
    """
    
    # 1. Carrega providers disponíveis (built-in + plugins)
    providers.load_builtins_once()
    providers.load_plugins_once()
    
    # 2. RESOLUÇÃO: Qual classe instanciar?
    if config.provider:
        # Seleção explícita por nome
        provider_class = router.resolve_provider(config.provider)
    else:
        # Seleção automática por model_id
        provider_class = router.resolve(config.model_id)
    
    # 3. PREPARAÇÃO: Parâmetros com defaults
    kwargs = _kwargs_with_environment_defaults(
        config.model_id or config.provider or "",
        config.provider_kwargs
    )
    
    if config.model_id:
        kwargs["model_id"] = config.model_id
    
    # 4. INSTANCIAÇÃO: Cria o produto concreto
    try:
        model = provider_class(**kwargs)
        return model
    except (ValueError, TypeError) as e:
        raise InferenceConfigError(
            f"Failed to create provider {provider_class.__name__}: {e}"
        ) from e

# ============================================
# HELPER: Adiciona defaults de ambiente
# ============================================
def _kwargs_with_environment_defaults(
    model_id: str,
    kwargs: dict[str, Any]
) -> dict[str, Any]:
    """Adiciona API keys de variáveis de ambiente"""
    resolved = dict(kwargs)
    
    if "api_key" not in resolved:
        # Tenta buscar de GEMINI_API_KEY, OPENAI_API_KEY, etc.
        env_vars_by_provider = {
            "gemini": ("GEMINI_API_KEY", "LANGEXTRACT_API_KEY"),
            "gpt": ("OPENAI_API_KEY", "LANGEXTRACT_API_KEY"),
        }
        
        for provider_prefix, env_vars in env_vars_by_provider.items():
            if provider_prefix in model_id.lower():
                for env_var in env_vars:
                    api_key = os.getenv(env_var)
                    if api_key:
                        resolved["api_key"] = api_key
                        break
    
    return resolved
```

### **PRODUCTS - Implementações concretas (providers)**

```python
# ============================================
# ARQUIVO: langextract/providers/gemini.py
# ============================================
class GeminiLanguageModel(BaseLanguageModel):
    """Provider concreto para Gemini"""
    
    def __init__(self, model_id: str = 'gemini-2.5-flash', api_key: str = None, **kwargs):
        self.model_id = model_id
        self.api_key = api_key
        self._client = genai.Client(api_key=api_key)
    
    def infer(self, batch_prompts: Sequence[str], **kwargs):
        """Implementação específica de inferência"""
        for prompt in batch_prompts:
            response = self._client.models.generate_content(
                model=self.model_id,
                contents=prompt
            )
            yield [ScoredOutput(score=1.0, output=response.text)]

# ============================================
# ARQUIVO: langextract/providers/openai.py
# ============================================
class OpenAILanguageModel(BaseLanguageModel):
    """Provider concreto para OpenAI"""
    
    def __init__(self, model_id: str = 'gpt-4o', api_key: str = None, **kwargs):
        self.model_id = model_id
        self.api_key = api_key
        self._client = openai.OpenAI(api_key=api_key)
    
    def infer(self, batch_prompts: Sequence[str], **kwargs):
        """Implementação específica de inferência"""
        for prompt in batch_prompts:
            response = self._client.chat.completions.create(
                model=self.model_id,
                messages=[{"role": "user", "content": prompt}]
            )
            yield [ScoredOutput(score=1.0, output=response.choices[0].message.content)]
```

### **INTERFACE COMUM (Product)**

```python
# ============================================
# ARQUIVO: langextract/core/base_model.py
# ============================================
class BaseLanguageModel(ABC):
    """Interface comum para todos os providers"""
    
    @abstractmethod
    def infer(self, batch_prompts: Sequence[str], **kwargs) -> Iterator[Sequence[ScoredOutput]]:
        """Método que todo provider deve implementar"""
        pass
```

---

## 📊 **3. COMPARAÇÃO LADO A LADO**

### **Tabela Comparativa**

| Aspecto | Factory Clássico (GoF) | LangExtract Factory |
|---------|------------------------|---------------------|
| **Factory Method** | `create_animal()` (abstrato) | `create_model()` (função) |
| **Creators** | `DogFactory`, `CatFactory` | Não há classes creator separadas |
| **Products** | `Dog`, `Cat` | `GeminiLanguageModel`, `OpenAILanguageModel` |
| **Interface Product** | `Animal` | `BaseLanguageModel` |
| **Resolução** | Polimorfismo (qual creator?) | Router + Registry (qual provider?) |
| **Configuração** | Hardcoded na factory | `ModelConfig` dataclass |
| **Defaults** | Não tem | `_kwargs_with_environment_defaults()` |
| **Extensibilidade** | Criar nova subclasse Creator | Plugin system (entry points) |

---

## 🎯 **4. MAPEAMENTO CONCEITUAL**

### **Elementos do Padrão GoF → LangExtract**

```
┌─────────────────────────────────────────────────────────────┐
│  PADRÃO GoF              →    LANGEXTRACT                   │
├─────────────────────────────────────────────────────────────┤
│  Creator (abstract)      →    Não existe (função direta)    │
│  factoryMethod()         →    create_model()                │
│  ConcreteCreator         →    Não existe (resolvido pelo    │
│                               Router)                        │
│  Product (interface)     →    BaseLanguageModel             │
│  ConcreteProductA        →    GeminiLanguageModel           │
│  ConcreteProductB        →    OpenAILanguageModel           │
│  ConcreteProductC        →    OllamaLanguageModel           │
└─────────────────────────────────────────────────────────────┘
```

### **Fluxo de Decisão**

```
┌─────────────────────┐
│ Cliente chama:      │
│ create_model()      │
└──────────┬──────────┘
           │
           ▼
┌─────────────────────┐
│ ModelConfig         │
│ - model_id          │ ──┐
│ - provider          │   │
└─────────────────────┘   │
                          │
                          ▼
                 ┌─────────────────┐
                 │ Router.resolve()│
                 │ (Registry)      │
                 └────────┬────────┘
                          │
        ┌─────────────────┼─────────────────┐
        ▼                 ▼                 ▼
┌──────────────┐  ┌──────────────┐  ┌──────────────┐
│ Gemini       │  │ OpenAI       │  │ Ollama       │
│ LanguageModel│  │ LanguageModel│  │ LanguageModel│
└──────────────┘  └──────────────┘  └──────────────┘
```

---

## 💡 **5. VARIAÇÕES DO PADRÃO**

### **A) Simple Factory (o que LangExtract usa)**

```python
# ❌ NÃO TEM hierarquia de Creators
# ✅ TEM função factory simples

def create_model(config: ModelConfig) -> BaseLanguageModel:
    # Decisão baseada em config
    if config.provider == "gemini":
        return GeminiLanguageModel(...)
    elif config.provider == "openai":
        return OpenAILanguageModel(...)
    # ...
```

**Característica:** Uma única função decide qual classe instanciar.

### **B) Factory Method (GoF clássico)**

```python
# ✅ TEM hierarquia de Creators
# ✅ TEM método abstrato

class ModelFactory(ABC):
    @abstractmethod
    def create_model(self) -> BaseLanguageModel:
        pass

class GeminiFactory(ModelFactory):
    def create_model(self) -> BaseLanguageModel:
        return GeminiLanguageModel(...)

class OpenAIFactory(ModelFactory):
    def create_model(self) -> BaseLanguageModel:
        return OpenAILanguageModel(...)
```

**Característica:** Hierarquia de classes factory.

### **C) Abstract Factory (múltiplas famílias)**

```python
class LLMProviderFactory(ABC):
    @abstractmethod
    def create_language_model(self) -> BaseLanguageModel:
        pass
    
    @abstractmethod
    def create_embedder(self) -> BaseEmbedder:
        pass
    
    @abstractmethod
    def create_tokenizer(self) -> BaseTokenizer:
        pass

class GeminiProviderFactory(LLMProviderFactory):
    # Cria família completa de produtos Gemini
    ...
```

**Característica:** Cria famílias de produtos relacionados.

---

## ✅ **6. VEREDITO: QUAL VARIAÇÃO É USADA?**

### **LangExtract implementa: Simple Factory + Registry Pattern**

```
Simple Factory:
    ✅ Função create_model() centralizada
    ✅ Não há hierarquia de creators
    ✅ Decisão interna sobre qual classe criar

Registry Pattern (adicional):
    ✅ Router mapeia model_id → provider_class
    ✅ Permite registro dinâmico de providers
    ✅ Extensível via plugins
```

### **Por que não é Factory Method puro?**

1. **Não há classes Creator**: Apenas uma função `create_model()`
2. **Não usa polimorfismo de Creator**: Usa Router para resolver
3. **Mais simples**: Adequado para o caso de uso

### **Por que não é Abstract Factory?**

1. **Não cria famílias de produtos**: Só cria `BaseLanguageModel`
2. **Não há múltiplos métodos factory relacionados**

---

## 📈 **7. VANTAGENS DA IMPLEMENTAÇÃO DO LANGEXTRACT**

| Vantagem | Descrição |
|----------|-----------|
| **Simplicidade** | Uma função, não hierarquia de classes |
| **Flexibilidade** | ModelConfig aceita model_id OU provider explícito |
| **Extensibilidade** | Plugin system permite adicionar providers sem modificar código |
| **Defaults inteligentes** | API keys de variáveis de ambiente |
| **Separação de responsabilidades** | Router cuida da resolução, Factory cuida da instanciação |
| **Type Safety** | Retorna tipo abstrato `BaseLanguageModel` |

---

## 🎓 **8. EXEMPLO DE USO COMPARADO**

### **Factory Clássico**

```python
# Cliente precisa saber qual factory usar
factory = DogFactory()  # ← Cliente decide
animal = factory.create_animal()
print(animal.speak())
```

### **LangExtract Factory**

```python
# Cliente só passa configuração, factory decide
config = ModelConfig(model_id="gemini-2.5-flash")
model = create_model(config)  # ← Factory decide internamente
result = model.infer(["Hello"])
```

---

## 🔧 **9. CÓDIGO COMPLETO DE COMPARAÇÃO**

### **Factory Clássico GoF - Versão Completa**

```python
from abc import ABC, abstractmethod

# ========== PRODUCTS ==========
class Vehicle(ABC):
    @abstractmethod
    def drive(self) -> str:
        pass

class Car(Vehicle):
    def drive(self) -> str:
        return "Driving a car 🚗"

class Truck(Vehicle):
    def drive(self) -> str:
        return "Driving a truck 🚚"

# ========== CREATORS ==========
class VehicleFactory(ABC):
    @abstractmethod
    def create_vehicle(self) -> Vehicle:
        """Factory Method"""
        pass
    
    def deliver(self) -> str:
        """Operação que usa o factory method"""
        vehicle = self.create_vehicle()
        return vehicle.drive()

class CarFactory(VehicleFactory):
    def create_vehicle(self) -> Vehicle:
        return Car()

class TruckFactory(VehicleFactory):
    def create_vehicle(self) -> Vehicle:
        return Truck()

# ========== USO ==========
def main():
    # Cliente escolhe factory
    factory: VehicleFactory = CarFactory()
    print(factory.deliver())  # Driving a car 🚗
    
    factory = TruckFactory()
    print(factory.deliver())  # Driving a truck 🚚

if __name__ == "__main__":
    main()
```

### **LangExtract Factory - Versão Simplificada**

```python
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, Type

# ========== PRODUCT INTERFACE ==========
class BaseLanguageModel(ABC):
    @abstractmethod
    def infer(self, prompt: str) -> str:
        pass

# ========== CONCRETE PRODUCTS ==========
class GeminiLanguageModel(BaseLanguageModel):
    def __init__(self, model_id: str, api_key: str):
        self.model_id = model_id
        self.api_key = api_key
    
    def infer(self, prompt: str) -> str:
        return f"Gemini ({self.model_id}): {prompt}"

class OpenAILanguageModel(BaseLanguageModel):
    def __init__(self, model_id: str, api_key: str):
        self.model_id = model_id
        self.api_key = api_key
    
    def infer(self, prompt: str) -> str:
        return f"OpenAI ({self.model_id}): {prompt}"

# ========== REGISTRY (Router) ==========
class ProviderRegistry:
    _providers: Dict[str, Type[BaseLanguageModel]] = {
        "gemini": GeminiLanguageModel,
        "openai": OpenAILanguageModel,
    }
    
    @classmethod
    def resolve(cls, model_id: str) -> Type[BaseLanguageModel]:
        if "gemini" in model_id.lower():
            return cls._providers["gemini"]
        elif "gpt" in model_id.lower():
            return cls._providers["openai"]
        raise ValueError(f"Unknown model: {model_id}")

# ========== CONFIGURATION ==========
@dataclass
class ModelConfig:
    model_id: str
    api_key: str

# ========== SIMPLE FACTORY ==========
def create_model(config: ModelConfig) -> BaseLanguageModel:
    """Factory function - Simple Factory Pattern"""
    
    # 1. Resolve qual classe usar
    provider_class = ProviderRegistry.resolve(config.model_id)
    
    # 2. Instancia com parâmetros
    model = provider_class(
        model_id=config.model_id,
        api_key=config.api_key
    )
    
    return model

# ========== USO ==========
def main():
    # Cliente não sabe qual provider será usado
    config = ModelConfig(
        model_id="gemini-2.5-flash",
        api_key="fake-key"
    )
    
    model = create_model(config)  # ← Factory decide
    print(model.infer("Hello!"))  # Gemini (gemini-2.5-flash): Hello!
    
    # Trocar provider é transparente
    config = ModelConfig(
        model_id="gpt-4o",
        api_key="fake-key"
    )
    
    model = create_model(config)
    print(model.infer("Hello!"))  # OpenAI (gpt-4o): Hello!

if __name__ == "__main__":
    main()
```




# 🔄 Iterator Pattern - Análise LangExtract
## Comparação: chunking.py vs Padrão GoF

---

## 📚 **1. PADRÃO ITERATOR (GoF) - RESUMO**

### **Definição**
> Fornece uma maneira de acessar sequencialmente elementos de uma coleção sem expor sua representação interna.

### **Estrutura Clássica**

```python
from abc import ABC, abstractmethod

# Iterator Interface
class Iterator(ABC):
    @abstractmethod
    def __next__(self):
        pass
    
    @abstractmethod
    def has_next(self) -> bool:
        pass

# Aggregate Interface
class Iterable(ABC):
    @abstractmethod
    def __iter__(self) -> Iterator:
        pass

# Concrete Iterator
class ListIterator(Iterator):
    def __init__(self, collection: list):
        self._collection = collection
        self._position = 0
    
    def __next__(self):
        if self.has_next():
            item = self._collection[self._position]
            self._position += 1
            return item
        raise StopIteration
    
    def has_next(self) -> bool:
        return self._position < len(self._collection)

# Concrete Aggregate
class MyList(Iterable):
    def __init__(self):
        self._items = []
    
    def add(self, item):
        self._items.append(item)
    
    def __iter__(self) -> Iterator:
        return ListIterator(self._items)

# USO
my_list = MyList()
my_list.add("A")
my_list.add("B")

for item in my_list:  # Python chama __iter__() e __next__()
    print(item)
```

---

## 🔍 **2. ITERATOR NO LANGEXTRACT (chunking.py)**

### **A) ChunkIterator - Divide documentos em chunks**

```python
# ============================================
# ARQUIVO: langextract/chunking.py
# ============================================

class ChunkIterator:
    """Itera chunks de texto tokenizado respeitando max_char_buffer"""
    
    def __init__(
        self,
        text: str | TokenizedText,
        max_char_buffer: int,
        document: Document | None = None,
    ):
        if isinstance(text, str):
            text = TokenizedText(text=text)
        
        self.tokenized_text = text
        self.max_char_buffer = max_char_buffer
        self.sentence_iter = SentenceIterator(self.tokenized_text)
        self.broken_sentence = False
        self.document = document if document else Document(text=text.text)
    
    def __iter__(self) -> Iterator[TextChunk]:
        """Protocolo Iterator: retorna self"""
        return self
    
    def __next__(self) -> TextChunk:
        """Protocolo Iterator: retorna próximo chunk"""
        sentence = next(self.sentence_iter)  # Pode lançar StopIteration
        
        # Inicializa chunk com primeiro token
        curr_chunk = create_token_interval(
            sentence.start_index,
            sentence.start_index + 1
        )
        
        # Se token excede buffer, retorna só ele
        if self._tokens_exceed_buffer(curr_chunk):
            self.sentence_iter = SentenceIterator(
                self.tokenized_text,
                curr_token_pos=sentence.start_index + 1
            )
            self.broken_sentence = True
            return TextChunk(token_interval=curr_chunk, document=self.document)
        
        # Adiciona tokens até atingir max_char_buffer
        start_of_new_line = -1
        for token_index in range(curr_chunk.start_index, sentence.end_index):
            if self.tokenized_text.tokens[token_index].first_token_after_newline:
                start_of_new_line = token_index
            
            test_chunk = create_token_interval(
                curr_chunk.start_index,
                token_index + 1
            )
            
            if self._tokens_exceed_buffer(test_chunk):
                # Quebra em newline se possível
                if start_of_new_line > curr_chunk.start_index:
                    curr_chunk = create_token_interval(
                        curr_chunk.start_index,
                        start_of_new_line
                    )
                
                self.sentence_iter = SentenceIterator(
                    self.tokenized_text,
                    curr_token_pos=curr_chunk.end_index
                )
                self.broken_sentence = True
                return TextChunk(token_interval=curr_chunk, document=self.document)
            else:
                curr_chunk = test_chunk
        
        # Tenta adicionar sentenças completas se couber
        if not self.broken_sentence:
            for sentence in self.sentence_iter:
                test_chunk = create_token_interval(
                    curr_chunk.start_index,
                    sentence.end_index
                )
                
                if self._tokens_exceed_buffer(test_chunk):
                    self.sentence_iter = SentenceIterator(
                        self.tokenized_text,
                        curr_token_pos=curr_chunk.end_index
                    )
                    return TextChunk(token_interval=curr_chunk, document=self.document)
                else:
                    curr_chunk = test_chunk
        
        self.broken_sentence = False
        return TextChunk(token_interval=curr_chunk, document=self.document)
    
    def _tokens_exceed_buffer(self, token_interval: TokenInterval) -> bool:
        """Verifica se intervalo excede buffer máximo"""
        char_interval = get_char_interval(self.tokenized_text, token_interval)
        return (char_interval.end_pos - char_interval.start_pos) > self.max_char_buffer
```

### **B) SentenceIterator - Itera por sentenças**

```python
class SentenceIterator:
    """Itera através de sentenças em texto tokenizado"""
    
    def __init__(
        self,
        tokenized_text: TokenizedText,
        curr_token_pos: int = 0,
    ):
        self.tokenized_text = tokenized_text
        self.token_len = len(tokenized_text.tokens)
        
        if curr_token_pos < 0 or curr_token_pos > self.token_len:
            raise IndexError(f"Invalid token position: {curr_token_pos}")
        
        self.curr_token_pos = curr_token_pos
    
    def __iter__(self) -> Iterator[TokenInterval]:
        """Protocolo Iterator: retorna self"""
        return self
    
    def __next__(self) -> TokenInterval:
        """Protocolo Iterator: retorna próxima sentença"""
        if self.curr_token_pos == self.token_len:
            raise StopIteration
        
        # Localiza range da sentença contendo token atual
        sentence_range = tokenizer.find_sentence_range(
            self.tokenized_text.text,
            self.tokenized_text.tokens,
            self.curr_token_pos,
        )
        
        # Ajusta para começar da posição atual
        sentence_range = create_token_interval(
            self.curr_token_pos,
            sentence_range.end_index
        )
        
        self.curr_token_pos = sentence_range.end_index
        return sentence_range
```

---

## 📊 **3. COMPARAÇÃO DIRETA**

| Aspecto | Iterator Clássico | LangExtract ChunkIterator |
|---------|-------------------|---------------------------|
| **`__iter__()`** | ✅ Retorna Iterator | ✅ `return self` |
| **`__next__()`** | ✅ Retorna próximo item | ✅ Retorna `TextChunk` |
| **`StopIteration`** | ✅ Lança quando acabar | ✅ Delegado para `SentenceIterator` |
| **Estado interno** | `_position` (índice) | `sentence_iter`, `broken_sentence` |
| **Lazy evaluation** | ✅ SIM | ✅ SIM (processa sob demanda) |
| **Iteração aninhada** | ❌ Não | ✅ SIM (`ChunkIterator` usa `SentenceIterator`) |
| **Lógica complexa** | ❌ Simples (index++) | ✅ Complexa (buffer, newlines, sentenças) |

---

## 🎯 **4. MAPEAMENTO CONCEITUAL**

```
┌──────────────────────────────────────────────────────────┐
│  PADRÃO GoF          →    LANGEXTRACT                    │
├──────────────────────────────────────────────────────────┤
│  Iterator            →    ChunkIterator                  │
│  __iter__()          →    return self                    │
│  __next__()          →    Retorna TextChunk              │
│  has_next()          →    Implícito (StopIteration)      │
│  Aggregate           →    TokenizedText (coleção)        │
│  Item                →    TextChunk (elemento)           │
│  Estado (_position)  →    sentence_iter + broken_sentence│
└──────────────────────────────────────────────────────────┘
```

---

## 💡 **5. DIFERENÇAS PRINCIPAIS**

### **A) Iterator Clássico: Simples**
```python
class SimpleIterator:
    def __init__(self, data):
        self.data = data
        self.index = 0  # ← Estado simples
    
    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration
        item = self.data[self.index]
        self.index += 1  # ← Incremento simples
        return item
```

### **B) ChunkIterator: Complexo**
```python
class ChunkIterator:
    def __init__(self, text, max_char_buffer):
        self.tokenized_text = text
        self.max_char_buffer = max_char_buffer
        self.sentence_iter = SentenceIterator(...)  # ← Iterator aninhado
        self.broken_sentence = False  # ← Estado adicional
    
    def __next__(self):
        # 1. Pega sentença
        sentence = next(self.sentence_iter)
        
        # 2. Lógica complexa: buffer, newlines, tokens
        # ... 50+ linhas de lógica
        
        # 3. Retorna chunk otimizado
        return TextChunk(...)
```

**Por quê?**
- Precisa respeitar limite de caracteres (`max_char_buffer`)
- Prefere quebrar em newlines quando possível
- Agrupa sentenças completas se couberem no buffer
- Trata tokens gigantes que excedem buffer

---

## 🔧 **6. EXEMPLO DE USO COMPARADO**

### **Iterator Clássico**

```python
# Cria coleção iterável
numbers = MyList()
numbers.add(1)
numbers.add(2)
numbers.add(3)

# Itera
for num in numbers:
    print(num)

# Output: 1, 2, 3
```

### **ChunkIterator no LangExtract**

```python
# Texto longo
text = """
This is a very long document that needs to be split into chunks
for processing by an LLM with limited context window.
Each chunk should be around 200 characters maximum.
"""

# Cria iterator
chunk_iter = ChunkIterator(
    text=text,
    max_char_buffer=200,
    document=Document(text=text)
)

# Itera por chunks
for chunk in chunk_iter:
    print(f"Chunk: {chunk.chunk_text[:50]}...")
    print(f"Size: {len(chunk.chunk_text)} chars\n")

# Output:
# Chunk: This is a very long document that needs to be ...
# Size: 197 chars
#
# Chunk: for processing by an LLM with limited context...
# Size: 186 chars
```

---

## ✅ **7. CHECKLIST DE VERIFICAÇÃO**

| Característica do Iterator Pattern | ChunkIterator | SentenceIterator |
|-------------------------------------|---------------|------------------|
| ✅ Implementa `__iter__()` | ✅ SIM | ✅ SIM |
| ✅ Implementa `__next__()` | ✅ SIM | ✅ SIM |
| ✅ Lança `StopIteration` | ✅ SIM | ✅ SIM |
| ✅ Mantém estado interno | ✅ SIM | ✅ SIM |
| ✅ Lazy evaluation | ✅ SIM | ✅ SIM |
| ✅ Funciona com `for` loop | ✅ SIM | ✅ SIM |
| ✅ Esconde representação interna | ✅ SIM | ✅ SIM |

---

## 📈 **8. VANTAGENS DA IMPLEMENTAÇÃO**

| Vantagem | Descrição |
|----------|-----------|
| **Memória eficiente** | Não carrega documento inteiro na memória |
| **Streaming** | Processa chunks sob demanda |
| **Flexível** | Lógica complexa de chunking encapsulada |
| **Pythonic** | Usa protocolo nativo (`__iter__`, `__next__`) |
| **Composição** | `ChunkIterator` usa `SentenceIterator` internamente |
| **Reutilizável** | Pode iterar múltiplas vezes criando novo iterator |

---

## 🎓 **9. CÓDIGO EXECUTÁVEL COMPLETO**

### **A) Iterator Clássico (Simples)**

```python
from abc import ABC, abstractmethod

class Iterator(ABC):
    @abstractmethod
    def __next__(self):
        pass

class BookShelf:
    """Aggregate: Coleção de livros"""
    def __init__(self):
        self._books = []
    
    def add(self, book: str):
        self._books.append(book)
    
    def __iter__(self):
        """Retorna iterator"""
        return BookIterator(self._books)

class BookIterator:
    """Iterator: Percorre livros"""
    def __init__(self, books: list):
        self._books = books
        self._index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index >= len(self._books):
            raise StopIteration
        
        book = self._books[self._index]
        self._index += 1
        return book

# USO
shelf = BookShelf()
shelf.add("Python Design Patterns")
shelf.add("Clean Code")
shelf.add("Refactoring")

for book in shelf:
    print(f"📚 {book}")
```

### **B) ChunkIterator (Simplificado)**

```python
from dataclasses import dataclass

@dataclass
class Chunk:
    text: str
    start: int
    end: int

class TextChunker:
    """Iterator que divide texto em chunks de tamanho fixo"""
    
    def __init__(self, text: str, chunk_size: int):
        self.text = text
        self.chunk_size = chunk_size
        self.position = 0
    
    def __iter__(self):
        return self
    
    def __next__(self) -> Chunk:
        if self.position >= len(self.text):
            raise StopIteration
        
        start = self.position
        end = min(self.position + self.chunk_size, len(self.text))
        
        # Tenta quebrar em espaço para não partir palavras
        if end < len(self.text) and self.text[end] != ' ':
            # Procura último espaço antes do fim
            last_space = self.text.rfind(' ', start, end)
            if last_space > start:
                end = last_space + 1
        
        chunk_text = self.text[start:end]
        self.position = end
        
        return Chunk(text=chunk_text, start=start, end=end)

# USO
text = "The quick brown fox jumps over the lazy dog. Pack my box with five dozen liquor jugs."
chunker = TextChunker(text, chunk_size=30)

for i, chunk in enumerate(chunker, 1):
    print(f"Chunk {i}: '{chunk.text}' [{chunk.start}:{chunk.end}]")

# Output:
# Chunk 1: 'The quick brown fox jumps ' [0:26]
# Chunk 2: 'over the lazy dog. Pack my ' [26:53]
# Chunk 3: 'box with five dozen liquor ' [53:80]
# Chunk 4: 'jugs.' [80:85]
```

---

## 📝 **10. CONCLUSÃO**

### **✅ É Iterator Pattern? SIM!**

**ChunkIterator e SentenceIterator implementam PERFEITAMENTE o padrão Iterator:**

1. ✅ Protocolo Python (`__iter__`, `__next__`, `StopIteration`)
2. ✅ Acesso sequencial sem expor representação interna
3. ✅ Lazy evaluation (processa sob demanda)
4. ✅ Estado interno gerenciado pelo iterator
5. ✅ Permite múltiplas iterações independentes

### **Diferencial do LangExtract:**

- **Lógica de negócio complexa** embutida no iterator
- **Iterators compostos** (`ChunkIterator` usa `SentenceIterator`)
- **Otimizações** (newlines, buffer, sentenças completas)
- **Aplicação real** para processamento de LLMs

### **Classificação:**
🏆 **Iterator Pattern (GoF) - Implementação Avançada com Lógica de Domínio**




# 🎭 Facade Pattern - Análise LangExtract
## Comparação: extraction.py vs Padrão GoF

---

## 📚 **1. PADRÃO FACADE (GoF) - RESUMO**

### **Definição**
> Fornece uma interface unificada para um conjunto de interfaces em um subsistema. Facade define uma interface de nível mais alto que torna o subsistema mais fácil de usar.

### **Estrutura Clássica**

```python
# ============================================
# SUBSISTEMAS COMPLEXOS (múltiplas classes)
# ============================================

class SubsystemA:
    def operation_a1(self):
        return "SubsystemA: operação A1"
    
    def operation_a2(self):
        return "SubsystemA: operação A2"

class SubsystemB:
    def operation_b1(self):
        return "SubsystemB: operação B1"
    
    def operation_b2(self):
        return "SubsystemB: operação B2"

class SubsystemC:
    def operation_c1(self):
        return "SubsystemC: operação C1"

# ============================================
# FACADE - Interface simplificada
# ============================================

class Facade:
    """Esconde a complexidade dos subsistemas"""
    
    def __init__(self):
        self._subsystem_a = SubsystemA()
        self._subsystem_b = SubsystemB()
        self._subsystem_c = SubsystemC()
    
    def simple_operation(self):
        """Uma chamada simples que coordena subsistemas"""
        results = []
        results.append(self._subsystem_a.operation_a1())
        results.append(self._subsystem_b.operation_b1())
        results.append(self._subsystem_c.operation_c1())
        return "\n".join(results)

# ============================================
# USO - Cliente usa facade, não subsistemas
# ============================================

# ❌ SEM Facade (cliente lida com complexidade):
subsystem_a = SubsystemA()
subsystem_b = SubsystemB()
subsystem_c = SubsystemC()
result1 = subsystem_a.operation_a1()
result2 = subsystem_b.operation_b1()
result3 = subsystem_c.operation_c1()

# ✅ COM Facade (cliente usa interface simples):
facade = Facade()
result = facade.simple_operation()  # ← Tudo em uma chamada!
```

---

## 🔍 **2. FACADE NO LANGEXTRACT (extraction.py)**

### **A) Função `extract()` - A Facade Principal**

```python
# ============================================
# ARQUIVO: langextract/extraction.py
# ============================================

def extract(
    text_or_documents: Any,
    prompt_description: str | None = None,
    examples: Sequence[Any] | None = None,
    model_id: str = "gemini-2.5-flash",
    api_key: str | None = None,
    # ... muitos outros parâmetros opcionais
) -> Any:
    """
    FACADE: Interface simplificada para extração estruturada.
    
    Esconde a complexidade de:
    - Factory (criação de models)
    - Prompting (geração de prompts)
    - FormatHandler (parsing JSON/YAML)
    - Resolver (conversão string → Extraction)
    - Annotator (pipeline de extração)
    - Chunking (divisão de documentos)
    - Alignment (mapeamento texto → posições)
    """
    
    # ============================================
    # 1. VALIDAÇÃO
    # ============================================
    if not examples:
        raise ValueError("Examples are required...")
    
    if prompt_validation_level is not pv.PromptValidationLevel.OFF:
        report = pv.validate_prompt_alignment(examples=examples, ...)
        pv.handle_alignment_report(report, ...)
    
    # ============================================
    # 2. SUBSISTEMA: DOWNLOAD (se necessário)
    # ============================================
    if fetch_urls and isinstance(text_or_documents, str) and io.is_url(text_or_documents):
        text_or_documents = io.download_text_from_url(text_or_documents)
    
    # ============================================
    # 3. SUBSISTEMA: PROMPTING
    # ============================================
    prompt_template = prompting.PromptTemplateStructured(
        description=prompt_description
    )
    prompt_template.examples.extend(examples)
    
    # ============================================
    # 4. SUBSISTEMA: FACTORY (criação de model)
    # ============================================
    if model:
        language_model = model
    elif config:
        language_model = factory.create_model(
            config=config,
            examples=prompt_template.examples if use_schema_constraints else None,
            use_schema_constraints=use_schema_constraints,
            fence_output=fence_output,
        )
    else:
        # Cria config e usa factory
        config = factory.ModelConfig(
            model_id=model_id,
            provider_kwargs=filtered_kwargs
        )
        language_model = factory.create_model(config=config, ...)
    
    # ============================================
    # 5. SUBSISTEMA: FORMAT HANDLER
    # ============================================
    format_handler, remaining_params = fh.FormatHandler.from_resolver_params(
        resolver_params=resolver_params,
        base_format_type=format_type,
        base_use_fences=language_model.requires_fence_output,
        ...
    )
    
    # ============================================
    # 6. SUBSISTEMA: RESOLVER
    # ============================================
    res = resolver.Resolver(**effective_params)
    
    # ============================================
    # 7. SUBSISTEMA: ANNOTATOR (pipeline principal)
    # ============================================
    annotator = annotation.Annotator(
        language_model=language_model,
        prompt_template=prompt_template,
        format_handler=format_handler,
    )
    
    # ============================================
    # 8. EXECUÇÃO (coordena tudo)
    # ============================================
    if isinstance(text_or_documents, str):
        return annotator.annotate_text(
            text=text_or_documents,
            resolver=res,
            max_char_buffer=max_char_buffer,
            batch_length=batch_length,
            additional_context=additional_context,
            extraction_passes=extraction_passes,
            show_progress=show_progress,
            **alignment_kwargs,
        )
    else:
        return annotator.annotate_documents(
            documents=text_or_documents,
            resolver=res,
            max_char_buffer=max_char_buffer,
            batch_length=batch_length,
            extraction_passes=extraction_passes,
            show_progress=show_progress,
            **alignment_kwargs,
        )
```

### **B) Subsistemas Complexos que a Facade Esconde**

```python
# ============================================
# SUBSISTEMA 1: Factory (factory.py)
# ============================================
class ModelConfig:
    model_id: str
    provider: str | None
    provider_kwargs: dict

def create_model(config: ModelConfig, ...) -> BaseLanguageModel:
    # Lógica complexa de resolução de providers
    providers.load_builtins_once()
    providers.load_plugins_once()
    provider_class = router.resolve(config.model_id)
    kwargs = _kwargs_with_environment_defaults(...)
    return provider_class(**kwargs)

# ============================================
# SUBSISTEMA 2: Prompting (prompting.py)
# ============================================
class PromptTemplateStructured:
    description: str
    examples: list[ExampleData]

class QAPromptGenerator:
    def render(self, question: str, ...) -> str:
        # Constrói prompt complexo com exemplos
        ...

# ============================================
# SUBSISTEMA 3: FormatHandler (format_handler.py)
# ============================================
class FormatHandler:
    def parse_output(self, text: str) -> Sequence[Mapping]:
        # Parse JSON/YAML com fences, wrappers, etc.
        ...

# ============================================
# SUBSISTEMA 4: Resolver (resolver.py)
# ============================================
class Resolver:
    def resolve(self, input_text: str) -> Sequence[Extraction]:
        # Converte string → Extraction objects
        ...
    
    def align(self, extractions, source_text, ...) -> Iterator[Extraction]:
        # Alinha extrações com texto fonte
        ...

# ============================================
# SUBSISTEMA 5: Annotator (annotation.py)
# ============================================
class Annotator:
    def annotate_text(self, text: str, ...) -> AnnotatedDocument:
        # Pipeline completo: chunk → prompt → infer → resolve → align
        ...

# ============================================
# SUBSISTEMA 6: Chunking (chunking.py)
# ============================================
class ChunkIterator:
    def __next__(self) -> TextChunk:
        # Divide texto em chunks respeitando buffer
        ...
```

---

## 📊 **3. COMPARAÇÃO DIRETA**

| Aspecto | Facade Clássico | LangExtract `extract()` |
|---------|-----------------|-------------------------|
| **Interface única** | ✅ `simple_operation()` | ✅ `extract()` |
| **Esconde subsistemas** | ✅ SubsystemA, B, C | ✅ Factory, Prompting, Resolver, etc. |
| **Coordena operações** | ✅ Chama múltiplos subsistemas | ✅ Orquestra 6+ subsistemas |
| **Simplifica uso** | ✅ Cliente não vê complexidade | ✅ `lx.extract(text="...", model_id="...")` |
| **Defaults inteligentes** | ❌ Não tem | ✅ SIM (model_id padrão, etc.) |
| **Validação** | ❌ Não tem | ✅ SIM (examples, prompts) |
| **Flexibilidade** | ❌ Interface fixa | ✅ Muitos parâmetros opcionais |

---

## 🎯 **4. MAPEAMENTO CONCEITUAL**

```
┌────────────────────────────────────────────────────────┐
│  PADRÃO GoF          →    LANGEXTRACT                  │
├────────────────────────────────────────────────────────┤
│  Facade              →    extract()                    │
│  SubsystemA          →    factory (create_model)       │
│  SubsystemB          →    prompting (render prompts)   │
│  SubsystemC          →    resolver (parse output)      │
│  SubsystemD          →    annotator (pipeline)         │
│  SubsystemE          →    format_handler (JSON/YAML)   │
│  SubsystemF          →    chunking (divide texto)      │
│  simple_operation()  →    extract(text, model_id, ...) │
│  Cliente             →    Usuário final                │
└────────────────────────────────────────────────────────┘
```

### **Diagrama de Dependências**

```
┌─────────────────────────────────────────┐
│  CLIENTE                                │
│  (Usuário do LangExtract)               │
└────────────────┬────────────────────────┘
                 │
                 │ lx.extract(text="...", model_id="gemini")
                 │
                 ▼
┌─────────────────────────────────────────┐
│  FACADE: extract()                      │  ◄─── Interface Simples
│  (extraction.py)                        │
└────────┬────────────────────────────────┘
         │
         │ Coordena subsistemas ↓
         │
    ┌────┴───────────────────────────────┐
    │                                    │
    ▼                                    ▼
┌─────────────┐                  ┌─────────────┐
│  Factory    │                  │  Prompting  │
│  (criar     │                  │  (gerar     │
│   modelo)   │                  │   prompts)  │
└──────┬──────┘                  └──────┬──────┘
       │                                │
       ▼                                ▼
┌─────────────┐                  ┌─────────────┐
│  Router     │                  │  Format     │
│  (resolve   │                  │  Handler    │
│   provider) │                  │  (parse)    │
└─────────────┘                  └──────┬──────┘
                                        │
                                        ▼
                                 ┌─────────────┐
                                 │  Resolver   │
                                 │  (align)    │
                                 └──────┬──────┘
                                        │
                                        ▼
                                 ┌─────────────┐
                                 │  Annotator  │
                                 │  (pipeline) │
                                 └──────┬──────┘
                                        │
                                        ▼
                                 ┌─────────────┐
                                 │  Chunking   │
                                 │  (iterator) │
                                 └─────────────┘
```

---

## 💡 **5. COMPARAÇÃO: COM vs SEM FACADE**

### **❌ SEM Facade (Cliente faz tudo manualmente)**

```python
# Cliente precisa conhecer TODOS os subsistemas
from langextract import factory, prompting, resolver, annotation
from langextract import io, providers
from langextract.core import format_handler as fh

# 1. Download se necessário
if io.is_url(text):
    text = io.download_text_from_url(text)

# 2. Criar prompt template
prompt_template = prompting.PromptTemplateStructured(
    description="Extract entities"
)
prompt_template.examples.extend(examples)

# 3. Configurar e criar model
providers.load_builtins_once()
providers.load_plugins_once()
config = factory.ModelConfig(
    model_id="gemini-2.5-flash",
    provider_kwargs={"api_key": "..."}
)
language_model = factory.create_model(config)

# 4. Criar format handler
format_handler = fh.FormatHandler(
    format_type=data.FormatType.JSON,
    use_fences=language_model.requires_fence_output,
    use_wrapper=True,
    wrapper_key="extractions"
)

# 5. Criar resolver
res = resolver.Resolver(format_handler=format_handler)

# 6. Criar annotator
annotator = annotation.Annotator(
    language_model=language_model,
    prompt_template=prompt_template,
    format_handler=format_handler
)

# 7. Executar anotação
result = annotator.annotate_text(
    text=text,
    resolver=res,
    max_char_buffer=1000,
    batch_length=10,
    extraction_passes=1,
    show_progress=True
)

# 😰 Muito complexo para o usuário!
```

### **✅ COM Facade (Interface Simples)**

```python
import langextract as lx

# Uma linha faz tudo!
result = lx.extract(
    text="Your document text here...",
    prompt_description="Extract entities",
    examples=[...],
    model_id="gemini-2.5-flash"
)

# 😊 Simples e direto!
```

---

## 🔧 **6. EXEMPLO PRÁTICO COMPLETO**

### **A) Facade Clássico (Sistema de Home Theater)**

```python
# ============================================
# SUBSISTEMAS COMPLEXOS
# ============================================

class Amplifier:
    def on(self): return "Amplifier ligado"
    def set_volume(self, level): return f"Volume: {level}"
    def off(self): return "Amplifier desligado"

class DVDPlayer:
    def on(self): return "DVD ligado"
    def play(self, movie): return f"Reproduzindo: {movie}"
    def stop(self): return "DVD parado"
    def off(self): return "DVD desligado"

class Projector:
    def on(self): return "Projetor ligado"
    def wide_screen_mode(self): return "Modo widescreen"
    def off(self): return "Projetor desligado"

class Lights:
    def dim(self, level): return f"Luzes: {level}%"

# ============================================
# FACADE - Interface Simples
# ============================================

class HomeTheaterFacade:
    """Simplifica operação do home theater"""
    
    def __init__(self):
        self.amp = Amplifier()
        self.dvd = DVDPlayer()
        self.projector = Projector()
        self.lights = Lights()
    
    def watch_movie(self, movie: str):
        """Uma chamada, múltiplas operações coordenadas"""
        print("Preparando para assistir filme...")
        print(self.lights.dim(10))
        print(self.projector.on())
        print(self.projector.wide_screen_mode())
        print(self.amp.on())
        print(self.amp.set_volume(5))
        print(self.dvd.on())
        print(self.dvd.play(movie))
    
    def end_movie(self):
        """Desliga tudo de uma vez"""
        print("Finalizando filme...")
        print(self.dvd.stop())
        print(self.dvd.off())
        print(self.amp.off())
        print(self.projector.off())
        print(self.lights.dim(100))

# ============================================
# USO
# ============================================

theater = HomeTheaterFacade()
theater.watch_movie("Matrix")  # ← Simples!
# ... assistir filme ...
theater.end_movie()  # ← Simples!
```

### **B) LangExtract Facade (Simplificado)**

```python
# ============================================
# VERSÃO SIMPLIFICADA MOSTRANDO SUBSISTEMAS
# ============================================

class SimpleLangExtract:
    """Facade simplificada do LangExtract"""
    
    def extract(
        self,
        text: str,
        prompt: str,
        examples: list,
        model_id: str = "gemini-2.5-flash"
    ):
        """Interface simples que esconde 6 subsistemas"""
        
        # SUBSISTEMA 1: Prompting
        prompt_template = self._create_prompt(prompt, examples)
        
        # SUBSISTEMA 2: Factory
        model = self._create_model(model_id)
        
        # SUBSISTEMA 3: Format Handler
        format_handler = self._create_format_handler(model)
        
        # SUBSISTEMA 4: Resolver
        resolver = self._create_resolver(format_handler)
        
        # SUBSISTEMA 5: Annotator
        annotator = self._create_annotator(model, prompt_template, format_handler)
        
        # SUBSISTEMA 6: Execução (usa Chunking internamente)
        result = annotator.annotate_text(text, resolver)
        
        return result
    
    def _create_prompt(self, description, examples):
        # Lógica complexa de prompting
        return PromptTemplate(description, examples)
    
    def _create_model(self, model_id):
        # Lógica complexa de factory + router + providers
        return ModelFactory.create(model_id)
    
    def _create_format_handler(self, model):
        # Lógica de parsing JSON/YAML
        return FormatHandler(model.format_type)
    
    def _create_resolver(self, format_handler):
        # Lógica de conversão string → objetos
        return Resolver(format_handler)
    
    def _create_annotator(self, model, prompt, format_handler):
        # Lógica de pipeline de extração
        return Annotator(model, prompt, format_handler)

# ============================================
# USO
# ============================================

lx = SimpleLangExtract()

result = lx.extract(
    text="Romeo loves Juliet",
    prompt="Extract relationships",
    examples=[...],
    model_id="gemini-2.5-flash"
)  # ← Interface simples esconde toda complexidade!
```

---

## ✅ **7. CHECKLIST DE VERIFICAÇÃO**

| Característica do Facade Pattern | `extract()` |
|-----------------------------------|-------------|
| ✅ Interface unificada de alto nível | ✅ SIM |
| ✅ Esconde complexidade de subsistemas | ✅ SIM (6+ subsistemas) |
| ✅ Coordena múltiplas operações | ✅ SIM (factory → prompting → resolver → annotator) |
| ✅ Simplifica uso para cliente | ✅ SIM (1 linha vs 50+ linhas) |
| ✅ Subsistemas ainda acessíveis | ✅ SIM (usuários avançados podem usar direto) |
| ✅ Reduz acoplamento | ✅ SIM (cliente não depende de subsistemas) |
| ✅ Facilita manutenção | ✅ SIM (mudanças internas não afetam API) |

---

## 📈 **8. VANTAGENS DA IMPLEMENTAÇÃO**

| Vantagem | Descrição |
|----------|-----------|
| **Simplicidade** | API de 1 linha para operação complexa |
| **Defaults inteligentes** | `model_id="gemini-2.5-flash"`, parâmetros opcionais |
| **Flexibilidade** | Usuários avançados podem passar `model`, `config`, etc. |
| **Validação** | Valida inputs antes de processar |
| **Documentação clara** | Docstring explica todos os parâmetros |
| **Backward compatibility** | Warnings para parâmetros deprecados |
| **Extensibilidade** | Subsistemas podem evoluir independentemente |



# 🏗️ Builder Pattern - Análise LangExtract
## Comparação: prompting.py vs Padrão GoF

---

## 📚 **1. PADRÃO BUILDER (GoF) - RESUMO**

### **Definição**
> Separa a construção de um objeto complexo de sua representação, permitindo que o mesmo processo de construção crie diferentes representações.

### **Estrutura Clássica**

```python
from abc import ABC, abstractmethod

# ============================================
# PRODUTO - Objeto complexo a ser construído
# ============================================

class Pizza:
    def __init__(self):
        self.dough = None
        self.sauce = None
        self.topping = None
    
    def __str__(self):
        return f"Pizza: {self.dough}, {self.sauce}, {self.topping}"

# ============================================
# BUILDER - Interface abstrata
# ============================================

class PizzaBuilder(ABC):
    def __init__(self):
        self.pizza = Pizza()
    
    @abstractmethod
    def build_dough(self):
        pass
    
    @abstractmethod
    def build_sauce(self):
        pass
    
    @abstractmethod
    def build_topping(self):
        pass
    
    def get_pizza(self) -> Pizza:
        return self.pizza

# ============================================
# CONCRETE BUILDERS
# ============================================

class MargheritaBuilder(PizzaBuilder):
    def build_dough(self):
        self.pizza.dough = "Massa fina"
        return self
    
    def build_sauce(self):
        self.pizza.sauce = "Molho de tomate"
        return self
    
    def build_topping(self):
        self.pizza.topping = "Mozzarella"
        return self

class PepperoniBuilder(PizzaBuilder):
    def build_dough(self):
        self.pizza.dough = "Massa grossa"
        return self
    
    def build_sauce(self):
        self.pizza.sauce = "Molho picante"
        return self
    
    def build_topping(self):
        self.pizza.topping = "Pepperoni"
        return self

# ============================================
# DIRECTOR (opcional) - Controla construção
# ============================================

class PizzaDirector:
    def __init__(self, builder: PizzaBuilder):
        self._builder = builder
    
    def make_pizza(self) -> Pizza:
        """Controla a ordem de construção"""
        return (self._builder
                .build_dough()
                .build_sauce()
                .build_topping()
                .get_pizza())

# ============================================
# USO
# ============================================

# Com Director
builder = MargheritaBuilder()
director = PizzaDirector(builder)
pizza = director.make_pizza()
print(pizza)  # Pizza: Massa fina, Molho de tomate, Mozzarella

# Sem Director (construção manual)
builder = PepperoniBuilder()
pizza = builder.build_dough().build_sauce().build_topping().get_pizza()
print(pizza)  # Pizza: Massa grossa, Molho picante, Pepperoni
```

---

## 🔍 **2. BUILDER NO LANGEXTRACT (prompting.py)**

### **A) QAPromptGenerator - O Builder**

```python
# ============================================
# ARQUIVO: langextract/prompting.py
# ============================================

@dataclasses.dataclass
class QAPromptGenerator:
    """
    BUILDER: Constrói prompts complexos incrementalmente.
    
    O produto final é uma string de prompt formatada com:
    - Descrição/instruções
    - Contexto adicional (opcional)
    - Exemplos few-shot formatados
    - Pergunta
    - Prefixos Q:/A:
    """
    
    # Componentes do prompt (configuração)
    template: PromptTemplateStructured
    format_handler: FormatHandler
    examples_heading: str = "Examples"
    question_prefix: str = "Q: "
    answer_prefix: str = "A: "
    
    def format_example_as_text(self, example: ExampleData) -> str:
        """
        PASSO 1: Formata um único exemplo.
        
        Constrói string com:
        - Pergunta (Q: texto do exemplo)
        - Resposta (A: extrações formatadas)
        """
        question = example.text
        answer = self.format_handler.format_extraction_example(example.extractions)
        
        return "\n".join([
            f"{self.question_prefix}{question}",
            f"{self.answer_prefix}{answer}\n",
        ])
    
    def render(self, question: str, additional_context: str | None = None) -> str:
        """
        MÉTODO PRINCIPAL: Constrói o prompt completo passo a passo.
        
        Builder Pattern aplicado:
        1. Adiciona descrição
        2. Adiciona contexto (se houver)
        3. Adiciona heading de exemplos
        4. Adiciona cada exemplo formatado
        5. Adiciona pergunta
        6. Adiciona prefixo de resposta
        
        Retorna: String de prompt completa
        """
        prompt_lines: list[str] = []
        
        # PASSO 1: Descrição/Instruções
        prompt_lines.append(f"{self.template.description}\n")
        
        # PASSO 2: Contexto adicional (opcional)
        if additional_context:
            prompt_lines.append(f"{additional_context}\n")
        
        # PASSO 3: Seção de exemplos
        if self.template.examples:
            prompt_lines.append(self.examples_heading)
            
            # PASSO 4: Cada exemplo formatado
            for ex in self.template.examples:
                prompt_lines.append(self.format_example_as_text(ex))
        
        # PASSO 5: Pergunta atual
        prompt_lines.append(f"{self.question_prefix}{question}")
        
        # PASSO 6: Prefixo de resposta
        prompt_lines.append(self.answer_prefix)
        
        # PRODUTO FINAL: String completa
        return "\n".join(prompt_lines)
    
    def __str__(self) -> str:
        """Renderiza com pergunta vazia (para visualização)"""
        return self.render("")
```

### **B) PromptTemplateStructured - Configuração do Builder**

```python
@dataclasses.dataclass
class PromptTemplateStructured:
    """
    Dados de entrada para o Builder.
    
    Armazena:
    - description: Instruções para o LLM
    - examples: Lista de exemplos few-shot
    """
    description: str
    examples: list[ExampleData] = dataclasses.field(default_factory=list)
```

### **C) Produto Final - String de Prompt**

```python
# EXEMPLO DE PROMPT CONSTRUÍDO:
"""
Extract characters, emotions, and relationships in order of appearance.
Use exact text for extractions. Do not paraphrase or overlap entities.

Examples
Q: ROMEO. But soft! What light through yonder window breaks?
A: ```json
{
  "extractions": [
    {
      "character": "ROMEO",
      "character_attributes": {"emotional_state": "wonder"}
    },
    {
      "emotion": "But soft!",
      "emotion_attributes": {"feeling": "gentle awe"}
    }
  ]
}
```

Q: Lady Juliet gazed longingly at the stars, her heart aching for Romeo
A:
"""
# ← Builder construiu isso incrementalmente!
```

---

## 📊 **3. COMPARAÇÃO DIRETA**

| Aspecto | Builder Clássico | QAPromptGenerator |
|---------|------------------|-------------------|
| **Produto complexo** | `Pizza` (objeto) | `str` (prompt completo) |
| **Construção incremental** | ✅ Métodos build_*() | ✅ `prompt_lines.append()` |
| **Passos separados** | ✅ dough → sauce → topping | ✅ description → context → examples → question |
| **Ordem importa** | ✅ SIM | ✅ SIM (descrição primeiro, resposta por último) |
| **Fluent interface** | ✅ `return self` | ❌ Não (usa lista interna) |
| **Configurável** | ✅ Diferentes builders | ✅ Parâmetros (prefixes, headings) |
| **Produto final** | `get_pizza()` | `render()` retorna string |
| **Director** | ✅ `PizzaDirector` | ❌ Não tem (render() já é o "director") |

---

## 🎯 **4. MAPEAMENTO CONCEITUAL**

```
┌─────────────────────────────────────────────────────────┐
│  PADRÃO GoF          →    LANGEXTRACT                   │
├─────────────────────────────────────────────────────────┤
│  Builder             →    QAPromptGenerator             │
│  Product             →    str (prompt completo)         │
│  build_dough()       →    Adiciona description          │
│  build_sauce()       →    Adiciona context              │
│  build_topping()     →    Adiciona examples             │
│  build_extra()       →    Adiciona question + prefix    │
│  get_product()       →    render() retorna string       │
│  Director            →    Não tem (render() coordena)   │
│  Configuration       →    PromptTemplateStructured      │
└─────────────────────────────────────────────────────────┘
```

### **Fluxo de Construção**

```
┌─────────────────────┐
│  Cliente            │
│  (Annotator)        │
└──────────┬──────────┘
           │
           │ prompt_generator.render(question)
           │
           ▼
┌─────────────────────────────────────────┐
│  QAPromptGenerator (Builder)            │
│  render() método coordena construção:   │
└──────────┬──────────────────────────────┘
           │
           ├─ 1. prompt_lines.append(description)
           │
           ├─ 2. if context: prompt_lines.append(context)
           │
           ├─ 3. prompt_lines.append(examples_heading)
           │
           ├─ 4. for ex in examples:
           │      prompt_lines.append(format_example_as_text(ex))
           │
           ├─ 5. prompt_lines.append(question_prefix + question)
           │
           └─ 6. prompt_lines.append(answer_prefix)
           │
           ▼
    ┌──────────────────┐
    │  "\n".join(...)  │  ◄─── Produto Final (String)
    └──────────────────┘
```

---

## 💡 **5. DIFERENÇAS PRINCIPAIS**

### **A) Builder Clássico: Múltiplos Métodos**

```python
class PizzaBuilder:
    def __init__(self):
        self.pizza = Pizza()
    
    def build_dough(self):
        self.pizza.dough = "..."  # ← Método 1
        return self
    
    def build_sauce(self):
        self.pizza.sauce = "..."  # ← Método 2
        return self
    
    def build_topping(self):
        self.pizza.topping = "..." # ← Método 3
        return self
    
    def get_pizza(self) -> Pizza:
        return self.pizza  # ← Retorna produto

# Uso: Construção explícita
pizza = (builder
         .build_dough()
         .build_sauce()
         .build_topping()
         .get_pizza())
```

### **B) QAPromptGenerator: Método Único de Construção**

```python
class QAPromptGenerator:
    def render(self, question: str, additional_context: str | None = None) -> str:
        """Único método que coordena TODA construção"""
        prompt_lines = []
        
        # PASSO 1
        prompt_lines.append(self.template.description)
        
        # PASSO 2 (condicional)
        if additional_context:
            prompt_lines.append(additional_context)
        
        # PASSO 3
        if self.template.examples:
            prompt_lines.append(self.examples_heading)
        
        # PASSO 4 (loop)
        for ex in self.template.examples:
            prompt_lines.append(self.format_example_as_text(ex))
        
        # PASSO 5
        prompt_lines.append(f"{self.question_prefix}{question}")
        
        # PASSO 6
        prompt_lines.append(self.answer_prefix)
        
        # PRODUTO FINAL
        return "\n".join(prompt_lines)

# Uso: Uma chamada
prompt = generator.render(question="Extract entities from...")
```

**Por que a diferença?**
- Builder clássico: Flexibilidade máxima (cliente controla ordem)
- QAPromptGenerator: Ordem fixa faz sentido (descrição sempre primeiro)
- Prompts têm estrutura previsível (não precisam de fluent interface)

---

## 🔧 **6. EXEMPLO PRÁTICO COMPLETO**

### **A) Builder Clássico (Construção de Email)**

```python
# ============================================
# PRODUTO
# ============================================

class Email:
    def __init__(self):
        self.to = None
        self.subject = None
        self.body = None
        self.attachments = []
    
    def __str__(self):
        return f"To: {self.to}\nSubject: {self.subject}\n\n{self.body}"

# ============================================
# BUILDER
# ============================================

class EmailBuilder:
    def __init__(self):
        self.email = Email()
    
    def to(self, recipient: str):
        self.email.to = recipient
        return self
    
    def subject(self, subject: str):
        self.email.subject = subject
        return self
    
    def body(self, body: str):
        self.email.body = body
        return self
    
    def attach(self, filename: str):
        self.email.attachments.append(filename)
        return self
    
    def build(self) -> Email:
        return self.email

# ============================================
# USO - Fluent Interface
# ============================================

email = (EmailBuilder()
         .to("user@example.com")
         .subject("Hello")
         .body("How are you?")
         .attach("report.pdf")
         .build())

print(email)
```

### **B) QAPromptGenerator (Real do LangExtract)**

```python
# ============================================
# CONFIGURAÇÃO
# ============================================

from dataclasses import dataclass

@dataclass
class ExampleData:
    text: str
    extractions: list

@dataclass
class PromptTemplateStructured:
    description: str
    examples: list[ExampleData]

# ============================================
# BUILDER
# ============================================

class QAPromptGenerator:
    def __init__(
        self,
        template: PromptTemplateStructured,
        examples_heading: str = "Examples",
        question_prefix: str = "Q: ",
        answer_prefix: str = "A: "
    ):
        self.template = template
        self.examples_heading = examples_heading
        self.question_prefix = question_prefix
        self.answer_prefix = answer_prefix
    
    def format_example_as_text(self, example: ExampleData) -> str:
        """Formata um exemplo como Q:/A:"""
        return (
            f"{self.question_prefix}{example.text}\n"
            f"{self.answer_prefix}{example.extractions}\n"
        )
    
    def render(self, question: str, additional_context: str | None = None) -> str:
        """Constrói prompt completo"""
        lines = []
        
        # 1. Descrição
        lines.append(f"{self.template.description}\n")
        
        # 2. Contexto (opcional)
        if additional_context:
            lines.append(f"{additional_context}\n")
        
        # 3. Exemplos
        if self.template.examples:
            lines.append(self.examples_heading)
            for ex in self.template.examples:
                lines.append(self.format_example_as_text(ex))
        
        # 4. Pergunta
        lines.append(f"{self.question_prefix}{question}")
        
        # 5. Prefixo resposta
        lines.append(self.answer_prefix)
        
        return "\n".join(lines)

# ============================================
# USO
# ============================================

template = PromptTemplateStructured(
    description="Extract entities from text:",
    examples=[
        ExampleData(
            text="Alice loves Bob",
            extractions=["Alice", "Bob", "loves"]
        )
    ]
)

generator = QAPromptGenerator(template)
prompt = generator.render(
    question="Romeo sees Juliet",
    additional_context="Focus on names"
)

print(prompt)
# Output:
# Extract entities from text:
#
# Focus on names
#
# Examples
# Q: Alice loves Bob
# A: ["Alice", "Bob", "loves"]
#
# Q: Romeo sees Juliet
# A:
```

---

## ✅ **7. CHECKLIST DE VERIFICAÇÃO**

| Característica do Builder Pattern | QAPromptGenerator |
|------------------------------------|-------------------|
| ✅ Constrói objeto complexo | ✅ SIM (prompt multi-seção) |
| ✅ Construção incremental | ✅ SIM (append linha por linha) |
| ✅ Separa construção de representação | ✅ SIM (render() vs template data) |
| ✅ Passos bem definidos | ✅ SIM (6 passos claros) |
| ✅ Ordem de construção controlada | ✅ SIM (descrição → exemplos → questão) |
| ✅ Produto complexo final | ✅ SIM (string formatada) |
| ❌ Fluent interface (return self) | ❌ NÃO (usa lista interna) |
| ❌ Classe Director separada | ❌ NÃO (render() já é o director) |

---

## 📈 **8. VANTAGENS DA IMPLEMENTAÇÃO**

| Vantagem | Descrição |
|----------|-----------|
| **Ordem garantida** | Descrição sempre primeiro, resposta sempre última |
| **Configurável** | Prefixes, headings customizáveis |
| **Reutilizável** | Mesmo generator para múltiplas perguntas |
| **Separação de concerns** | Template (dados) vs Generator (construção) |
| **Testável** | Pode testar cada passo individualmente |
| **Legibilidade** | Código sequencial claro |

---

## 🎯 **9. VARIAÇÕES DO PADRÃO**

### **A) Builder Clássico (GoF)**
```python
# Múltiplos métodos build_*()
# Fluent interface (return self)
# get_product() explícito

builder = EmailBuilder()
email = (builder
         .to("...")
         .subject("...")
         .body("...")
         .build())  # ← get_product()
```

### **B) Telescoping Constructor (Anti-pattern)**
```python
# ❌ ANTIPADRÃO - Muitos parâmetros
prompt = create_prompt(
    description="...",
    context="...",
    examples=[...],
    examples_heading="...",
    question_prefix="...",
    answer_prefix="...",
    question="...",
)
# Difícil de ler e manter!
```

### **C) QAPromptGenerator (Builder Simplificado)**
```python
# ✅ Builder interno
# Método único render()
# Ordem fixa de passos

generator = QAPromptGenerator(template)
prompt = generator.render(question="...")  # ← Tudo em uma chamada
```

**Por que QAPromptGenerator é Builder?**
- Constrói produto complexo (prompt) passo a passo
- Separa configuração (template) de construção (render)
- Encapsula lógica de formatação
- Ordem de passos bem definida





# 🎯 Strategy Pattern - Análise LangExtract
## Comparação: base_model.py + Implementações (gemini.py, openai.py, ollama.py) vs Padrão GoF

---

## 📚 **1. PADRÃO STRATEGY (GoF) - RESUMO**

### **Definição**
> Define uma família de algoritmos, encapsula cada um deles e os torna intercambiáveis. Strategy permite que o algoritmo varie independentemente dos clientes que o utilizam.

### **Estrutura Clássica**

```python
from abc import ABC, abstractmethod

# ============================================
# STRATEGY - Interface abstrata
# ============================================

class CompressionStrategy(ABC):
    """Interface comum para todas as estratégias de compressão"""
    
    @abstractmethod
    def compress(self, data: str) -> bytes:
        """Comprime dados usando algoritmo específico"""
        pass

# ============================================
# CONCRETE STRATEGIES - Implementações
# ============================================

class ZipCompression(CompressionStrategy):
    """Estratégia concreta: Compressão ZIP"""
    
    def compress(self, data: str) -> bytes:
        # Algoritmo ZIP
        return f"ZIP({data})".encode()

class RarCompression(CompressionStrategy):
    """Estratégia concreta: Compressão RAR"""
    
    def compress(self, data: str) -> bytes:
        # Algoritmo RAR
        return f"RAR({data})".encode()

class GzipCompression(CompressionStrategy):
    """Estratégia concreta: Compressão GZIP"""
    
    def compress(self, data: str) -> bytes:
        # Algoritmo GZIP
        return f"GZIP({data})".encode()

# ============================================
# CONTEXT - Classe que usa Strategy
# ============================================

class FileCompressor:
    """Context: Usa estratégia de compressão"""
    
    def __init__(self, strategy: CompressionStrategy):
        self._strategy = strategy
    
    def set_strategy(self, strategy: CompressionStrategy):
        """Troca estratégia em tempo de execução"""
        self._strategy = strategy
    
    def compress_file(self, data: str) -> bytes:
        """Delega compressão para estratégia atual"""
        print(f"Usando estratégia: {self._strategy.__class__.__name__}")
        return self._strategy.compress(data)

# ============================================
# USO - Cliente escolhe estratégia
# ============================================

# Cliente escolhe ZIP
compressor = FileCompressor(ZipCompression())
result1 = compressor.compress_file("Hello World")
print(result1)  # b'ZIP(Hello World)'

# Troca para RAR em runtime
compressor.set_strategy(RarCompression())
result2 = compressor.compress_file("Hello World")
print(result2)  # b'RAR(Hello World)'

# Troca para GZIP
compressor.set_strategy(GzipCompression())
result3 = compressor.compress_file("Hello World")
print(result3)  # b'GZIP(Hello World)'
```

---

## 🔍 **2. STRATEGY NO LANGEXTRACT**

### **A) BaseLanguageModel - A Interface Strategy**

```python
# ============================================
# ARQUIVO: langextract/core/base_model.py
# ============================================

class BaseLanguageModel(abc.ABC):
    """
    STRATEGY INTERFACE: Interface abstrata para LLMs.
    
    Define contrato comum para todas as estratégias (providers):
    - infer() → Método principal (algoritmo intercambiável)
    - parse_output() → Parsing de resposta
    - merge_kwargs() → Configuração
    """
    
    def __init__(self, constraint: types.Constraint | None = None, **kwargs: Any):
        """Inicializa com configuração base"""
        self._constraint = constraint or types.Constraint()
        self._schema: schema.BaseSchema | None = None
        self._extra_kwargs: dict[str, Any] = kwargs.copy()
    
    @abc.abstractmethod
    def infer(
        self,
        batch_prompts: Sequence[str],
        **kwargs
    ) -> Iterator[Sequence[types.ScoredOutput]]:
        """
        ALGORITMO INTERCAMBIÁVEL: Inferência do LLM.
        
        Cada provider implementa SEU algoritmo:
        - Gemini: Usa google-genai SDK
        - OpenAI: Usa openai SDK
        - Ollama: Usa HTTP requests
        
        Args:
            batch_prompts: Lista de prompts
            **kwargs: Parâmetros específicos (temperature, etc.)
        
        Returns:
            Iterator de outputs com scores
        """
    
    def parse_output(self, output: str) -> Any:
        """Método comum (não abstrato) - pode ser sobrescrito"""
        format_type = getattr(self, 'format_type', types.FormatType.JSON)
        
        try:
            if format_type == types.FormatType.JSON:
                return json.loads(output)
            else:
                return yaml.safe_load(output)
        except Exception as e:
            raise ValueError(f'Failed to parse output: {str(e)}') from e
    
    def merge_kwargs(self, runtime_kwargs: Mapping[str, Any] | None = None) -> dict:
        """Método comum - merge de parâmetros"""
        base = getattr(self, '_extra_kwargs', {}) or {}
        incoming = dict(runtime_kwargs or {})
        return {**base, **incoming}
```

### **B) Concrete Strategy 1: GeminiLanguageModel**

```python
# ============================================
# ARQUIVO: langextract/providers/gemini.py
# ============================================

class GeminiLanguageModel(base_model.BaseLanguageModel):
    """
    CONCRETE STRATEGY 1: Implementação Google Gemini.
    
    Algoritmo específico:
    - Usa SDK google-genai
    - Suporta structured output (JSON schema)
    - Parallel processing com ThreadPoolExecutor
    """
    
    model_id: str = 'gemini-2.5-flash'
    api_key: str | None = None
    temperature: float = 0.0
    max_workers: int = 10
    
    def __init__(
        self,
        model_id: str = 'gemini-2.5-flash',
        api_key: str | None = None,
        temperature: float = 0.0,
        **kwargs
    ):
        """Inicializa cliente Gemini"""
        from google import genai
        
        self.model_id = model_id
        self.api_key = api_key
        self.temperature = temperature
        
        # Cliente Gemini SDK
        self._client = genai.Client(
            api_key=self.api_key,
            vertexai=False,
        )
        
        super().__init__(constraint=schema.Constraint())
    
    def infer(
        self,
        batch_prompts: Sequence[str],
        **kwargs
    ) -> Iterator[Sequence[core_types.ScoredOutput]]:
        """
        ALGORITMO GEMINI: Inferência via Google Genai SDK.
        
        Específico do Gemini:
        - Parallel processing (ThreadPoolExecutor)
        - response_schema para structured output
        - response_mime_type: application/json
        """
        config = {
            'temperature': kwargs.get('temperature', self.temperature),
        }
        
        # Gemini-specific: JSON schema support
        if self.gemini_schema:
            config['response_mime_type'] = 'application/json'
            config['response_schema'] = self.gemini_schema.schema_dict
        
        # Parallel processing (Gemini-specific optimization)
        if len(batch_prompts) > 1 and self.max_workers > 1:
            with concurrent.futures.ThreadPoolExecutor(
                max_workers=min(self.max_workers, len(batch_prompts))
            ) as executor:
                futures = {
                    executor.submit(self._process_single_prompt, p, config): i
                    for i, p in enumerate(batch_prompts)
                }
                
                results = [None] * len(batch_prompts)
                for future in concurrent.futures.as_completed(futures):
                    idx = futures[future]
                    results[idx] = future.result()
                
                for result in results:
                    yield [result]
        else:
            # Sequential processing
            for prompt in batch_prompts:
                result = self._process_single_prompt(prompt, config)
                yield [result]
    
    def _process_single_prompt(self, prompt: str, config: dict):
        """Processa um único prompt via Gemini API"""
        response = self._client.models.generate_content(
            model=self.model_id,
            contents=prompt,
            config=config
        )
        return core_types.ScoredOutput(score=1.0, output=response.text)
```

### **C) Concrete Strategy 2: OpenAILanguageModel**

```python
# ============================================
# ARQUIVO: langextract/providers/openai.py
# ============================================

class OpenAILanguageModel(base_model.BaseLanguageModel):
    """
    CONCRETE STRATEGY 2: Implementação OpenAI.
    
    Algoritmo específico:
    - Usa SDK openai
    - JSON mode (response_format)
    - Reasoning effort para o1/o3 models
    """
    
    model_id: str = 'gpt-4o-mini'
    api_key: str | None = None
    temperature: float | None = None
    
    def __init__(
        self,
        model_id: str = 'gpt-4o-mini',
        api_key: str | None = None,
        temperature: float | None = None,
        **kwargs
    ):
        """Inicializa cliente OpenAI"""
        import openai
        
        self.model_id = model_id
        self.api_key = api_key
        self.temperature = temperature
        
        # Cliente OpenAI SDK
        self._client = openai.OpenAI(api_key=self.api_key)
        
        super().__init__(constraint=schema.Constraint())
    
    def infer(
        self,
        batch_prompts: Sequence[str],
        **kwargs
    ) -> Iterator[Sequence[core_types.ScoredOutput]]:
        """
        ALGORITMO OPENAI: Inferência via OpenAI SDK.
        
        Específico do OpenAI:
        - Chat completions API
        - response_format: json_object
        - reasoning_effort para o1 models
        """
        config = {}
        
        if (temp := kwargs.get('temperature', self.temperature)) is not None:
            config['temperature'] = temp
        
        # OpenAI-specific: JSON mode
        if self.format_type == data.FormatType.JSON:
            config.setdefault('response_format', {'type': 'json_object'})
        
        # OpenAI-specific: Reasoning effort (o1 models)
        if 'reasoning_effort' in kwargs:
            config['reasoning'] = {'effort': kwargs['reasoning_effort']}
        
        # Parallel ou sequential
        for prompt in batch_prompts:
            result = self._process_single_prompt(prompt, config)
            yield [result]
    
    def _process_single_prompt(self, prompt: str, config: dict):
        """Processa um único prompt via OpenAI API"""
        messages = [
            {'role': 'system', 'content': 'You respond in JSON format.'},
            {'role': 'user', 'content': prompt}
        ]
        
        response = self._client.chat.completions.create(
            model=self.model_id,
            messages=messages,
            **config
        )
        
        output = response.choices[0].message.content
        return core_types.ScoredOutput(score=1.0, output=output)
    
    @property
    def requires_fence_output(self) -> bool:
        """OpenAI JSON mode retorna JSON puro (sem fences)"""
        if self.format_type == data.FormatType.JSON:
            return False  # ← Comportamento específico OpenAI
        return super().requires_fence_output
```

### **D) Concrete Strategy 3: OllamaLanguageModel**

```python
# ============================================
# ARQUIVO: langextract/providers/ollama.py
# ============================================

class OllamaLanguageModel(base_model.BaseLanguageModel):
    """
    CONCRETE STRATEGY 3: Implementação Ollama (local).
    
    Algoritmo específico:
    - Usa HTTP requests (não tem SDK oficial)
    - API local (localhost:11434)
    - JSON format mode
    """
    
    _model: str
    _model_url: str = 'http://localhost:11434'
    format_type: core_types.FormatType = core_types.FormatType.JSON
    
    def __init__(
        self,
        model_id: str,
        model_url: str = 'http://localhost:11434',
        format_type: core_types.FormatType = core_types.FormatType.JSON,
        **kwargs
    ):
        """Inicializa cliente Ollama (HTTP)"""
        self._model = model_id
        self._model_url = model_url
        self.format_type = format_type
        self._requests = requests  # HTTP client
        
        super().__init__(constraint=schema.Constraint())
    
    def infer(
        self,
        batch_prompts: Sequence[str],
        **kwargs
    ) -> Iterator[Sequence[core_types.ScoredOutput]]:
        """
        ALGORITMO OLLAMA: Inferência via HTTP requests.
        
        Específico do Ollama:
        - POST para /api/generate
        - format: 'json' no payload
        - Timeout configurável
        """
        for prompt in batch_prompts:
            try:
                response = self._ollama_query(
                    prompt=prompt,
                    model=self._model,
                    structured_output_format='json',
                    model_url=self._model_url,
                    **kwargs
                )
                yield [core_types.ScoredOutput(
                    score=1.0,
                    output=response['response']
                )]
            except Exception as e:
                raise exceptions.InferenceRuntimeError(
                    f'Ollama API error: {str(e)}'
                ) from e
    
    def _ollama_query(
        self,
        prompt: str,
        model: str,
        structured_output_format: str,
        model_url: str,
        **kwargs
    ) -> dict:
        """
        Query Ollama via HTTP POST.
        
        Ollama-specific implementation:
        - Endpoint: /api/generate
        - Payload: {model, prompt, format, options}
        """
        api_url = urljoin(model_url, 'api/generate')
        
        payload = {
            'model': model,
            'prompt': prompt,
            'stream': False,  # ← Ollama-specific
            'format': structured_output_format,  # ← Ollama JSON mode
            'options': {
                'temperature': kwargs.get('temperature', 0.1),
                'num_ctx': kwargs.get('num_ctx', 2048),
            }
        }
        
        # HTTP POST (sem SDK oficial)
        response = self._requests.post(
            api_url,
            json=payload,
            timeout=kwargs.get('timeout', 120)
        )
        
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            raise exceptions.InferenceConfigError(
                f"Model {model} not found. Try: ollama run {model}"
            )
        else:
            raise exceptions.InferenceRuntimeError(
                f'Ollama error: {response.status_code}'
            )
```

---

## 📊 **3. COMPARAÇÃO DIRETA**

| Aspecto | Strategy Clássico | LangExtract |
|---------|-------------------|-------------|
| **Interface abstrata** | `CompressionStrategy` | `BaseLanguageModel` |
| **Método abstrato** | `compress(data)` | `infer(batch_prompts, **kwargs)` |
| **Concrete Strategy 1** | `ZipCompression` | `GeminiLanguageModel` |
| **Concrete Strategy 2** | `RarCompression` | `OpenAILanguageModel` |
| **Concrete Strategy 3** | `GzipCompression` | `OllamaLanguageModel` |
| **Context** | `FileCompressor` | `Annotator` (usa model.infer()) |
| **Troca de estratégia** | `set_strategy()` | Factory cria estratégia correta |
| **Algoritmo varia** | ✅ SIM (ZIP vs RAR vs GZIP) | ✅ SIM (Gemini vs OpenAI vs Ollama) |
| **Interface comum** | ✅ SIM | ✅ SIM (infer + parse_output) |
| **Polimorfismo** | ✅ SIM | ✅ SIM (BaseLanguageModel) |

---

## 🎯 **4. MAPEAMENTO CONCEITUAL**

```
┌────────────────────────────────────────────────────────────┐
│  PADRÃO GoF                →    LANGEXTRACT                │
├────────────────────────────────────────────────────────────┤
│  Strategy (interface)      →    BaseLanguageModel          │
│  compress() abstrato       →    infer() abstrato           │
│  ConcreteStrategyA         →    GeminiLanguageModel        │
│  ConcreteStrategyB         →    OpenAILanguageModel        │
│  ConcreteStrategyC         →    OllamaLanguageModel        │
│  Context                   →    Annotator                  │
│  set_strategy()            →    create_model(config)       │
│  Cliente escolhe           →    ModelConfig especifica     │
└────────────────────────────────────────────────────────────┘
```

### **Fluxo de Execução**

```
┌──────────────────────┐
│  Cliente (extract)   │
└──────────┬───────────┘
           │
           │ ModelConfig(model_id="gemini-2.5-flash")
           │
           ▼
┌──────────────────────┐
│  Factory             │
│  create_model()      │
└──────────┬───────────┘
           │
           │ Cria estratégia apropriada
           │
           ▼
┌─────────────────────────────────────────────┐
│  BaseLanguageModel (Strategy Interface)     │
└──────────┬──────────────────────────────────┘
           │
           ├─── GeminiLanguageModel
           │    └─ infer() → google.genai SDK
           │
           ├─── OpenAILanguageModel
           │    └─ infer() → openai SDK
           │
           └─── OllamaLanguageModel
                └─ infer() → HTTP requests
           │
           ▼
┌──────────────────────┐
│  Annotator (Context) │
│  model.infer(prompt) │  ◄─── Usa estratégia polimorficamente
└──────────────────────┘
```

---

## 💡 **5. EXEMPLO PRÁTICO COMPLETO**

### **A) Strategy Clássico (Pagamento)**

```python
from abc import ABC, abstractmethod

# ============================================
# STRATEGY INTERFACE
# ============================================

class PaymentStrategy(ABC):
    @abstractmethod
    def pay(self, amount: float) -> str:
        pass

# ============================================
# CONCRETE STRATEGIES
# ============================================

class CreditCardPayment(PaymentStrategy):
    def __init__(self, card_number: str):
        self.card_number = card_number
    
    def pay(self, amount: float) -> str:
        return f"Paid ${amount} with Credit Card {self.card_number}"

class PayPalPayment(PaymentStrategy):
    def __init__(self, email: str):
        self.email = email
    
    def pay(self, amount: float) -> str:
        return f"Paid ${amount} with PayPal {self.email}"

class BitcoinPayment(PaymentStrategy):
    def __init__(self, wallet_address: str):
        self.wallet_address = wallet_address
    
    def pay(self, amount: float) -> str:
        return f"Paid ${amount} with Bitcoin {self.wallet_address}"

# ============================================
# CONTEXT
# ============================================

class ShoppingCart:
    def __init__(self):
        self._items = []
        self._payment_strategy: PaymentStrategy | None = None
    
    def add_item(self, item: str, price: float):
        self._items.append((item, price))
    
    def set_payment_strategy(self, strategy: PaymentStrategy):
        """Troca estratégia de pagamento"""
        self._payment_strategy = strategy
    
    def checkout(self) -> str:
        """Usa estratégia atual para pagamento"""
        total = sum(price for _, price in self._items)
        
        if not self._payment_strategy:
            raise ValueError("No payment strategy set")
        
        return self._payment_strategy.pay(total)

# ============================================
# USO
# ============================================

cart = ShoppingCart()
cart.add_item("Book", 29.99)
cart.add_item("Pen", 1.50)

# Cliente 1: Paga com cartão
cart.set_payment_strategy(CreditCardPayment("1234-5678-9012-3456"))
print(cart.checkout())  # Paid $31.49 with Credit Card 1234-...

# Cliente 2: Troca para PayPal
cart.set_payment_strategy(PayPalPayment("user@example.com"))
print(cart.checkout())  # Paid $31.49 with PayPal user@example.com

# Cliente 3: Troca para Bitcoin
cart.set_payment_strategy(BitcoinPayment("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa"))
print(cart.checkout())  # Paid $31.49 with Bitcoin 1A1z...
```

### **B) LangExtract Strategy (Real)**

```python
# ============================================
# USO NO LANGEXTRACT
# ============================================

import langextract as lx

# ============================================
# ESTRATÉGIA 1: Gemini
# ============================================

result1 = lx.extract(
    text_or_documents="Extract info from this text",
    model_id="gemini-2.5-flash",  # ← Factory escolhe GeminiLanguageModel
    api_key="...",
    prompt_description="Extract entities"
)
# Internamente:
# - Factory cria GeminiLanguageModel
# - Annotator chama model.infer()
# - GeminiLanguageModel.infer() usa google-genai SDK

# ============================================
# ESTRATÉGIA 2: OpenAI (troca em runtime)
# ============================================

result2 = lx.extract(
    text_or_documents="Same text",
    model_id="gpt-4o-mini",  # ← Factory escolhe OpenAILanguageModel
    api_key="...",
    prompt_description="Extract entities"
)
# Internamente:
# - Factory cria OpenAILanguageModel
# - Annotator chama model.infer()
# - OpenAILanguageModel.infer() usa openai SDK

# ============================================
# ESTRATÉGIA 3: Ollama (local)
# ============================================

result3 = lx.extract(
    text_or_documents="Same text",
    model_id="llama3.2:1b",  # ← Factory escolhe OllamaLanguageModel
    prompt_description="Extract entities"
)
# Internamente:
# - Factory cria OllamaLanguageModel
# - Annotator chama model.infer()
# - OllamaLanguageModel.infer() usa HTTP requests

# ============================================
# POLIMORFISMO EM AÇÃO
# ============================================

from langextract.core.base_model import BaseLanguageModel

def process_with_any_model(model: BaseLanguageModel, prompt: str):
    """Aceita QUALQUER estratégia (Gemini, OpenAI, Ollama)"""
    outputs = list(model.infer([prompt]))
    return outputs[0][0].output

# Funciona com todas as estratégias:
gemini_model = GeminiLanguageModel(api_key="...")
openai_model = OpenAILanguageModel(api_key="...")
ollama_model = OllamaLanguageModel(model_id="llama3.2:1b")

result_a = process_with_any_model(gemini_model, "Extract...")
result_b = process_with_any_model(openai_model, "Extract...")
result_c = process_with_any_model(ollama_model, "Extract...")
```

---

## ✅ **6. CHECKLIST DE VERIFICAÇÃO**

| Característica do Strategy Pattern | LangExtract |
|-------------------------------------|-------------|
| ✅ Interface abstrata com método abstrato | ✅ SIM (`BaseLanguageModel.infer()`) |
| ✅ Múltiplas implementações concretas | ✅ SIM (Gemini, OpenAI, Ollama) |
| ✅ Algoritmo varia entre estratégias | ✅ SIM (SDK vs HTTP, parallel vs sequential) |
| ✅ Interface comum para todas | ✅ SIM (`infer()` assinatura idêntica) |
| ✅ Polimorfismo em tempo de execução | ✅ SIM (Annotator usa BaseLanguageModel) |
| ✅ Estratégias intercambiáveis | ✅ SIM (Factory cria estratégia certa) |
| ✅ Cliente não conhece implementação | ✅ SIM (Annotator não sabe se é Gemini/OpenAI) |
| ✅ Facilita adição de novas estratégias | ✅ SIM (basta herdar BaseLanguageModel) |

---

## 📈 **7. COMPARAÇÃO DE ALGORITMOS**

### **Diferenças entre Estratégias**

| Aspecto | GeminiLanguageModel | OpenAILanguageModel | OllamaLanguageModel |
|---------|---------------------|---------------------|---------------------|
| **SDK** | google-genai | openai | requests (HTTP) |
| **Endpoint** | genai.Client | chat.completions | POST /api/generate |
| **JSON mode** | response_schema | response_format | format: 'json' |
| **Parallel** | ✅ ThreadPoolExecutor | ✅ ThreadPoolExecutor | ❌ Sequential |
| **Auth** | api_key ou Vertex AI | api_key | Opcional (local) |
| **Structured output** | ✅ Native (schema_dict) | ⚠️ JSON mode only | ⚠️ JSON mode only |
| **Timeout** | Configurável | Configurável | Configurável |
| **Streaming** | ❌ Não (batch) | ❌ Não (batch) | ❌ Não (stream=False) |

### **Exemplo de Diferença de Algoritmo**

```python
# ============================================
# GEMINI: Parallel processing
# ============================================

def infer(self, batch_prompts):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {
            executor.submit(self._process_single_prompt, p): i
            for i, p in enumerate(batch_prompts)
        }
        # Processa todos em paralelo
        for future in as_completed(futures):
            yield [future.result()]

# ============================================
# OPENAI: Parallel processing similar
# ============================================

def infer(self, batch_prompts):
    # Mesma estrutura de Gemini
    with ThreadPoolExecutor(max_workers=10) as executor:
        # ... parallel processing

# ============================================
# OLLAMA: Sequential processing
# ============================================

def infer(self, batch_prompts):
    # Sem parallel processing
    for prompt in batch_prompts:
        response = self._ollama_query(prompt, ...)
        yield [ScoredOutput(output=response['response'])]
```

---

## 🔧 **8. VANTAGENS DA IMPLEMENTAÇÃO**

| Vantagem | Descrição |
|----------|-----------|
| **Intercambiável** | Troca entre Gemini/OpenAI/Ollama sem mudar código cliente |
| **Extensível** | Nova estratégia = nova classe herda BaseLanguageModel |
| **Testável** | Pode criar MockLanguageModel para testes |
| **Encapsulamento** | Detalhes de API (SDK, HTTP) ocultos do cliente |
| **Polimórfico** | Annotator usa BaseLanguageModel (não importa qual) |
| **Configurável** | Cada estratégia tem seus parâmetros específicos |

---

## 🆕 **9. ADICIONANDO NOVA ESTRATÉGIA**

### **Como adicionar novo provider (ex: Anthropic Claude)**

```python
# ============================================
# NOVA ESTRATÉGIA: AnthropicLanguageModel
# ============================================

from langextract.core import base_model
from langextract.providers import router

@router.register("claude-*", priority=5)  # Registra padrão
class AnthropicLanguageModel(base_model.BaseLanguageModel):
    """
    NOVA CONCRETE STRATEGY: Anthropic Claude.
    
    Algoritmo específico:
    - Usa anthropic SDK
    - Messages API
    """
    
    model_id: str = "claude-3-5-sonnet-20241022"
    api_key: str | None = None
    
    def __init__(
        self,
        model_id: str = "claude-3-5-sonnet-20241022",
        api_key: str | None = None,
        **kwargs
    ):
        """Inicializa cliente Anthropic"""
        import anthropic
        
        self.model_id = model_id
        self.api_key = api_key
        
        # Cliente Anthropic SDK
        self._client = anthropic.Anthropic(api_key=api_key)
        
        super().__init__(constraint=schema.Constraint())
    
    def infer(
        self,
        batch_prompts: Sequence[str],
        **kwargs
    ) -> Iterator[Sequence[core_types.ScoredOutput]]:
        """
        ALGORITMO ANTHROPIC: Inferência via Anthropic SDK.
        
        Específico do Claude:
        - Messages API
        - System prompts
        - Temperature, max_tokens
        """
        for prompt in batch_prompts:
            response = self._client.messages.create(
                model=self.model_id,
                max_tokens=kwargs.get('max_tokens', 1024),
                temperature=kwargs.get('temperature', 0.0),
                messages=[{
                    "role": "user",
                    "content": prompt
                }]
            )
            
            output = response.content[0].text
            yield [core_types.ScoredOutput(score=1.0, output=output)]

# ============================================
# USO IMEDIATO
# ============================================

import langextract as lx

result = lx.extract(
    text_or_documents="Extract entities",
    model_id="claude-3-5-sonnet-20241022",  # ← Factory detecta automaticamente
    api_key="sk-ant-...",
    prompt_description="Extract names"
)
# Funciona! Factory cria AnthropicLanguageModel via @router.register
```

---

## 📝 **10. CONCLUSÃO**

### **✅ É Strategy Pattern? SIM! 100% GoF Compliant**

**Evidências:**

1. ✅ **Interface abstrata** - `BaseLanguageModel(ABC)` com `@abstractmethod infer()`
2. ✅ **Múltiplas estratégias** - Gemini, OpenAI, Ollama (+ extensível)
3. ✅ **Algoritmo intercambiável** - SDK google-genai vs openai vs HTTP
4. ✅ **Polimorfismo** - Annotator usa `BaseLanguageModel` (não conhece implementação)
5. ✅ **Troca em runtime** - Factory cria estratégia baseado em `model_id`
6. ✅ **Interface comum** - Todas implementam `infer()` com mesma assinatura
7. ✅ **Comportamento varia** - Parallel (Gemini) vs Sequential (Ollama)

### **Classificação:**
🏆 **Strategy Pattern (GoF) - Implementação Pura e Completa**

### **Por que é Strategy perfeito?**

```
┌────────────────────────────────────────────┐
│  OBJETIVO DO PADRÃO                        │
├────────────────────────────────────────────┤
│  "Permitir que algoritmo varie             │
│   independentemente dos clientes"          │
└────────────────────────────────────────────┘
                    │
                    │
                    ▼
┌────────────────────────────────────────────┐
│  LANGEXTRACT IMPLEMENTAÇÃO                 │
├────────────────────────────────────────────┤
│  ✅ Annotator (cliente) não sabe qual LLM  │
│  ✅ GeminiLanguageModel.infer() ≠          │
│      OpenAILanguageModel.infer()           │
│  ✅ Mesma interface, algoritmos diferentes │
│  ✅ Fácil adicionar nova estratégia        │
└────────────────────────────────────────────┘
```

### **Evidência Visual:**

```python
# ============================================
# SEM STRATEGY (❌ Código acoplado)
# ============================================

def extract(text, provider):
    if provider == "gemini":
        from google import genai
        client = genai.Client(api_key="...")
        response = client.models.generate_content(...)
        return response.text
    elif provider == "openai":
        import openai
        client = openai.OpenAI(api_key="...")
        response = client.chat.completions.create(...)
        return response.choices[0].message.content
    elif provider == "ollama":
        import requests
        response = requests.post("http://localhost:11434/api/generate", ...)
        return response.json()['response']
    # ❌ Difícil manter, adicionar novos providers

# ============================================
# COM STRATEGY (✅ Desacoplado)
# ============================================

def extract(text, model: BaseLanguageModel):
    """Aceita QUALQUER estratégia"""
    outputs = model.infer([text])
    return list(outputs)[0][0].output

# Uso com qualquer provider:
extract(text, GeminiLanguageModel(api_key="..."))
extract(text, OpenAILanguageModel(api_key="..."))
extract(text, OllamaLanguageModel(model_id="llama3.2"))
# ✅ Fácil adicionar AnthropicLanguageModel
```

